This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/dp.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from matplotlib import cm
from pydrake.all import (DynamicProgrammingOptions, FittedValueIteration,
                         LinearSystem, Simulator)

from underactuated import running_as_notebook

# The Grid World

The setup here is *almost* identical as the simplest version described in the notes.  The only difference is that this agent is allowed to move diagonally in a single step; this is slightly easier to code since I can have two actions (one for left/right, and another for up/down), and write the dynamics as the trivial linear system ${\bf x}[n+1] = {\bf u}[n].$  Only the value iteration code needs to know that the states and actions are actually restricted to the integers. I also add a very large cost when the action would be diagonal, so that it is never chosen.

The obstacle (pit of despair) is provided by the method below.  Play around with it!  The rest of the code is mostly to support visualization.

In [ ]:
def grid_world_example():
    time_step = 1
    # TODO(russt): Support discrete-time systems in the dynamic programming
    # code, and use this properly. For now, just cheat because I know how to
    # make the discrete system as a continuous that will be discretized.
    plant = LinearSystem(A=np.zeros((2, 2)),
                         B=np.eye(2),
                         C=np.eye(2),
                         D=np.zeros((2, 2)))
    simulator = Simulator(plant)
    options = DynamicProgrammingOptions()

    xbins = range(0, 21)
    ybins = range(0, 16)
    state_grid = [set(xbins), set(ybins)]

    input_grid = [set([-1, 0, 1]), set([-1, 0, 1])]

    goal = [2, 8]

    def obstacle(x):
        return x[0]>=6 and x[0]<=8 and x[1]>=4 and x[1]<=7

    [X, Y] = np.meshgrid(xbins, ybins)

    frames=[]
    def draw(iteration, mesh, cost_to_go, policy):
        J = np.reshape(cost_to_go, X.shape)
        artists = [ax.imshow(J, cmap=cm.jet)]
        artists += [
            ax.quiver(X,
                      Y,
                      np.reshape(policy[0], X.shape),
                      np.reshape(policy[1], Y.shape),
                      scale=1.4,
                      scale_units='x')
        ]
        frames.append(artists)

    if running_as_notebook:
        options.visualization_callback = draw

    def min_time_cost(context):
        x = context.get_continuous_state_vector().CopyToVector()
        x = np.round(x)
        state_cost = 1
        if obstacle(x):
            state_cost = 10
        if np.array_equal(x, goal):
            state_cost = 0
        u = plant.get_input_port(0).Eval(context)
        action_cost = np.linalg.norm(u, 1)
        if action_cost > 1:
            action_cost = 10
        return state_cost + action_cost

    cost_function = min_time_cost
    options.convergence_tol = .1;

    (fig,ax) = plt.subplots(figsize=(10,6))
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Cost-to-Go")

    policy, cost_to_go = FittedValueIteration(simulator, cost_function,
                                              state_grid, input_grid, time_step,
                                              options)

    draw('Final', None, cost_to_go, policy.get_output_values())

    ax.invert_yaxis()
    plt.colorbar(frames[-1][0])

    print("generating animation...")
    # create animation using the animate() function
    ani = animation.ArtistAnimation(fig,
                                    frames,
                                    interval=200,
                                    blit=True,
                                    repeat=False)
    plt.close('all')

    display(HTML(ani.to_jshtml()))

grid_world_example()

Your turn.  Change the cost.  Change the obstacles.